# Main program used to remove artifacts from magnetic resonance images

#### First we have to process the DICOM files in order to save it in .jgp format, add the required artifact and then save the transformed image as .jpg.

Preprocessing function has seven arguments:

    - path (string): the original DICOM folder.
    
    - BodyPart (string): it can be either 'brain' or 'knee'.
    
    - TypeArtifact (list of strings): the elements can be 'Motion', 'Ghosting', 'BiasField', 'Blur', 'Noise' and 'Spike'.
    
    - showImage (boolean): True if you want images to be shown, False if not.
    
    - saveImage (boolean): True if you want images to be saved, False if not.
    
    - multiprocess (boolean): True if you want to use multiprocessing, False if not. Default value: False.
    
    - num_process (int): number of processors you want to use. Default value: multiprocessing.cpu_count()

In [ ]:
import preprocessing
preprocessing.Preprocessing('fastMRI_brain_DICOM','brain',['Motion','Blur','BiasField','Spike','Noise','Ghosting'],False,True,True)

#### Now, we will divide the data into training, validation and testing sets. 

It's important to mention that this division will be made at the level of studies (patients), rather than at the level of images, so that we ensure that the different sets don't include images of the same brains. The classification of the studies in Training, Validation and Test will be the same for all the artifacts.

splitTrainValTest has just two arguments:

    - path (string): the original DICOM folder.
    
    - TypeArtifact (list of strings): the elements can be 'Motion', 'Ghosting', 'BiasField', 'Blur', 'Noise' and 'Spike'.
    
Note that the artifacts that we can include in the list are no more than those that we have included in the previous function.

In [3]:
import train_val_test
train_val_test.splitTrainValTest('fastMRI_brain_DICOM',['Motion','Blur','BiasField','Spike','Noise','Ghosting'])

#### We will now train our model.  

It will return the model already trained. Note that the architecture of the net must be changed in training_process.py file.

train_model has the following arguments:

    - type_artifact (string): it can be 'Motion', 'Ghosting', 'BiasField', 'Blur', 'Noise' and 'Spike'.
    
    - batch_size (int): the batch size that we want for our model.
    
    - learning_rate (float): the learning rate hyperparameter of our model.
    
    - num_epochs (int): the number of epoch that the training process will have.
    
    - filters_code (string): it represents :
         
         - the kernel size in case there are 64 filters for the first layer and 32 for the second. It can be '515', '915', '935' and '955'.
         
         - the number of layers of the model. It can be '95', '915', '9115' and '91115'.
         
         - the number of filters in each layer, with kernel size 9-1-5. It can be '16_8', '32_16', '64_32' and '128_64'.
      
Note that the model architecture used should be changed inside the file, and filters_code should have the name corresponding to it.
    

In [ ]:
import training_process
model = training_process.train_model('Blur', 16, 0.001, 40, '915')

#### We can now test any model we want.  

It will return a vector with the loss and the accuracies of the model as (loss, SSIM, PSNR, MS-SSIM). 

test_model has the following arguments:

    - type_artifact (string): it can be 'Motion', 'Ghosting', 'BiasField', 'Blur', 'Noise' and 'Spike'.
    
    - model (object of class ArtifactCNN): the model already loaded.
    
    - filters_code (string): the same as it was explained before.

In [ ]:
import training_process
import test_process
import torch
import torch.nn as nn
import torch.nn.functional as F

filters_code = '915'
artifact = 'Blur'

global ArtifactCNN
class ArtifactCNN(nn.Module):
    def __init__(self):
        super(ArtifactCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 64, kernel_size=9,padding=4)
        self.conv2 = nn.Conv2d(64, 32, kernel_size=1,padding=0)
        self.conv3 = nn.Conv2d(32, 1, kernel_size=5,padding=2)
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = self.conv3(x)
        return x
    
device = 'cuda:0' if torch.cuda.is_available() else 'cpu' 
model = ArtifactCNN().to(device)
model = torch.load(f'trained_models/modelo20k_{artifact}_{filters_code}.pth')
model.eval()

test_loss,test_ssim,test_psnr,test_ms_ssim = test_process.test_model(artifact,model, filters_code)
print(test_loss,test_ssim,test_psnr,test_ms_ssim)

#### We can plot the training data.

It will show and save the graph with the required data if we ask it to.

plots has the following artifacts:

    - list_artifacts (list of strings): the elements can be 'Motion', 'Ghosting', 'BiasField', 'Blur', 'Noise' and 'Spike'. They can be repeated if different models are considered
    
    - list_models (list of arrays): the elements can be '915', '935', '515'..., i.e. the name of the models that have been trained
    
    - metric (string): it can be 'MSE', 'SSIM' and 'PSNR'
    
    - show_fig (boolean): it indicates whether you want to show the image or not. The default value is True
    
    - save_fig (boolean): it indicates whether you want to save the image or not. The default value is True

In [ ]:
import training_plots
training_plots.plots(['Blur','Spike','Motion','Ghosting','BiasField','Noise'],['915','915','915','915','915','915'],'PSNR',show_fig=True,save_fig=True)

#### We can also plot the filters and feature maps of a certain model.
It will show the filters and the feature maps of a given model. 

function feature_maps has five parameters:

    - image_path (string): it is the path of the image we want
    
    - model (object of the class ArtifactCNN): the model we want to get the feature maps of
    
    - artifact (string): the artifact of the corresponding model
    
    - filters_code (string): it indicates the name of the corresponding model
    
    - save_fig (boolean): it indicates whether you want to save the image or not. The default value is True
    
while 'filters' has the four last described ones.


In [ ]:
import training_process
import post_training
import torch
import torch.nn as nn
import torch.nn.functional as F

filters_code = '915'
artifact = 'Noise'

global ArtifactCNN
class ArtifactCNN(nn.Module):
    def __init__(self):
        super(ArtifactCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 64, kernel_size=9,padding=4)
        self.conv2 = nn.Conv2d(64, 32, kernel_size=1,padding=0)
        self.conv3 = nn.Conv2d(32, 1, kernel_size=5,padding=2)
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = self.conv3(x)
        return x
    
device = 'cuda:0' if torch.cuda.is_available() else 'cpu' 
model = ArtifactCNN().to(device)
model = torch.load(f'trained_models/modelo20k_{artifact}_{filters_code}.pth')
model.eval()

post_training.filters(model, artifact, filters_code, save_fig = False)
post_training.feature_maps('fastMRI_brain_DICOM/Validation/106643169658/1015.jpg', model, artifact, filters_code, save_fig = False)